In [1]:
import os
import pandas as pd 
from functools import partial, reduce
import numpy as np 
import pickle
from sklearn.model_selection import train_test_split

import cv2 
import math 
from keras.optimizers import SGD 
from keras.callbacks import LearningRateScheduler
#=================== Environment variables ===================
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 

import tensorflow as tf 
import tensorflow.keras as keras
from tensorflow.keras.layers import Dense, Lambda, Conv3D, Dropout
from tensorflow.keras.layers import Activation, BatchNormalization
from tensorflow.keras.layers import Input, concatenate, Add, Flatten
from tensorflow.keras.layers import GlobalAveragePooling3D, GlobalMaxPooling3D, MaxPooling3D, AveragePooling3D
from tensorflow.keras.utils import plot_model
from tensorflow.keras.models import Model

Using TensorFlow backend.


In [76]:
def inception_module(x,
                     filters_1x1,
                     filters_3x3_reduce,
                     filters_3x3,
                     filters_5x5_reduce,
                     filters_5x5,
                     filters_pool_proj,
                     name=None):
    
    conv_1x1 = Conv3D(filters_1x1,  1, padding='same', activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)(x)
    
    conv_3x3 = Conv3D(filters_3x3_reduce,  1, padding='same', activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)(x)
    conv_3x3 = Conv3D(filters_3x3, 3, padding='same', activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)(conv_3x3)

    conv_5x5 = Conv3D(filters_5x5_reduce, 1, padding='same', activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)(x)
    conv_5x5 = Conv3D(filters_5x5, 5, padding='same', activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)(conv_5x5)

    pool_proj = MaxPooling3D(3, strides=(1, 1,1), padding='same')(x)
    pool_proj = Conv3D(filters_pool_proj, 1, padding='same', activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)(pool_proj)

    output = concatenate([conv_1x1, conv_3x3, conv_5x5, pool_proj], axis=4, name=name)
    
    return output

kernel_init = tf.keras.initializers.glorot_uniform()
bias_init = tf.keras.initializers.Constant(value=0.2)

In [106]:
input_layer = Input(shape=(53, 63, 52, 53))

x = Conv3D(64, 3, padding='same', strides=(2, 2, 2), activation='relu', name='conv_1_7x7/2', kernel_initializer=kernel_init, bias_initializer=bias_init)(input_layer)
x = MaxPooling3D(3, padding='same', strides=(2, 2,2), name='max_pool_1_3x3/2')(x)


In [107]:
x = inception_module(x,
                     filters_1x1=64,
                     filters_3x3_reduce=96,
                     filters_3x3=128,
                     filters_5x5_reduce=16,
                     filters_5x5=32,
                     filters_pool_proj=32,
                     name='inception_3a')
x = inception_module(x,
                     filters_1x1=128,
                     filters_3x3_reduce=128,
                     filters_3x3=192,
                     filters_5x5_reduce=32,
                     filters_5x5=96,
                     filters_pool_proj=64,
                     name='inception_3b')
x = MaxPooling3D(3, padding='same', strides=(2, 2,2), name='max_pool_3_3x3/2')(x)
x = inception_module(x,
                     filters_1x1=192,
                     filters_3x3_reduce=96,
                     filters_3x3=208,
                     filters_5x5_reduce=16,
                     filters_5x5=48,
                     filters_pool_proj=64,
                     name='inception_4a')
x1 = AveragePooling3D(5, strides=3)(x)
x1 = Conv3D(128, 1, padding='same', activation='relu')(x1)
x1 = Flatten()(x1)
x1 = Dense(1024, activation='relu')(x1)
x1 = Dropout(0.7)(x1)
x1 = Dense(5, activation='linear', name='auxilliary_output_1')(x1)

In [108]:
x = inception_module(x,
                     filters_1x1=160,
                     filters_3x3_reduce=112,
                     filters_3x3=224,
                     filters_5x5_reduce=24,
                     filters_5x5=64,
                     filters_pool_proj=64,
                     name='inception_4b')

x = inception_module(x,
                     filters_1x1=128,
                     filters_3x3_reduce=128,
                     filters_3x3=256,
                     filters_5x5_reduce=24,
                     filters_5x5=64,
                     filters_pool_proj=64,
                     name='inception_4c')

x = inception_module(x,
                     filters_1x1=112,
                     filters_3x3_reduce=144,
                     filters_3x3=288,
                     filters_5x5_reduce=32,
                     filters_5x5=64,
                     filters_pool_proj=64,
                     name='inception_4d')


x2 = AveragePooling3D(5, strides=3)(x)
x2 = Conv3D(128, 1, padding='same', activation='relu')(x2)
x2 = Flatten()(x2)
x2 = Dense(1024, activation='relu')(x2)
x2 = Dropout(0.7)(x2)
x2 = Dense(5, activation='linear', name='auxilliary_output_2')(x2)

In [109]:

x = inception_module(x,
                     filters_1x1=256,
                     filters_3x3_reduce=160,
                     filters_3x3=320,
                     filters_5x5_reduce=32,
                     filters_5x5=128,
                     filters_pool_proj=128,
                     name='inception_4e')

x = MaxPooling3D(3, padding='same', strides=(2, 2,2), name='max_pool_4_3x3/2')(x)

x = inception_module(x,
                     filters_1x1=256,
                     filters_3x3_reduce=160,
                     filters_3x3=320,
                     filters_5x5_reduce=32,
                     filters_5x5=128,
                     filters_pool_proj=128,
                     name='inception_5a')

x = inception_module(x,
                     filters_1x1=384,
                     filters_3x3_reduce=192,
                     filters_3x3=384,
                     filters_5x5_reduce=48,
                     filters_5x5=128,
                     filters_pool_proj=128,
                     name='inception_5b')

x = GlobalAveragePooling3D(name='avg_pool_5_3x3/1')(x)

x = Dropout(0.4)(x)

x = Dense(5, activation='linear', name='output')(x)
model = Model(input_layer, [x, x1, x2], name='inception_v1')
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_35 (InputLayer)           (None, 53, 63, 52, 5 0                                            
__________________________________________________________________________________________________
conv_1_7x7/2 (Conv3D)           (None, 27, 32, 26, 6 91648       input_35[0][0]                   
__________________________________________________________________________________________________
max_pool_1_3x3/2 (MaxPooling3D) (None, 14, 16, 13, 6 0           conv_1_7x7/2[0][0]               
__________________________________________________________________________________________________
conv3d_284 (Conv3D)             (None, 14, 16, 13, 9 6240        max_pool_1_3x3/2[0][0]           
__________________________________________________________________________________________________
conv3d_286